<a href="https://colab.research.google.com/github/GalEzra35/BI-Project/blob/hadar_branch/STTM_OOP_MENTAL_DISORDERS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [346]:
!pip install jsonpath_ng

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [347]:
import os
import csv
import json
import numpy as np 
import pandas as pd 
import sqlite3
import functools as ft
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from jsonpath_ng import parse
from enum import Enum
%matplotlib inline 

# Source to Target Mapping - OOP Approach for Single Table

### Data

In [348]:
with open("mental_disorders_50.json", "r") as file:
    data = json.loads(json.load(file))
data[:2]

[{'title': 'BPD and menstrual cycle',
  'selftext': 'Hi Iâ€™m 33 and have been diagnosed for 2 years. I struggle daily. But the worst is coming up to my period (sorry men) \nAny women on here also find that their cycle sends their emotions off into the stratosphere? I have been acting disgustingly toward my partner, I feel as though thereâ€™s no way I can even apologise and this happens each cycle. If you do find this happens to you, have you found anything which may help?',
  'created_utc': '1665162885',
  'over_18': 'FALSE',
  'subreddit': 'BPD'},
 {'title': "It's getting too much",
  'selftext': 'I just want to give into my impulses and finally let go of all self control. I just want to bash my head against a wall constantly and hurt myself. Getting help is so difficult and nothing works and I just want all the pain to end. Everything is so overwhelming and controlling my symptoms is exhausting and everyone turning on me for fake reasons. Everything sucks',
  'created_utc': '1658755

###  Abstract Base Calss (ABC) for mutual methods

In [349]:
class Interface(ABC):

    @abstractmethod
    def get_data_by_field(self, field_name):
        """Fetch the data by given feild name """

    @abstractmethod
    def get_data_by_id(self, id):
        """Fetch the data by given ID  """
  
    @abstractmethod
    def get(self):
        """Fetch all data """

### Transform Operations
inherithed from Enum - class that automatic enumrate the variables

In [350]:
class TransformMask(Enum):
    # add here any masks you want 
    CLEAN_STRING = ".strip().lower().replace('â€™','`')" 
    CAPITAL_LETTER = ".strip().lower().title().replace('â€™','`')"
    CAPITAL_STRING = ".strip().upper()"

### Database Class - Define Common Properties for Source, Target, Mapping

In [351]:
class Database:
    def __init__(self):
        self.db= {"source":[],
                  "destination":[],
                  "transform":[],
            "mapping": []  }

        self.add_source(1,'title',"str")
        self.add_source(2,'selftext',"str")
        self.add_source(3,'over_18',"str")
        self.add_source(4,'subreddit',"str")
        

        self.add_destination(1,"title","str","fact")
        self.add_destination(2,"selftext","str","fact")
        self.add_destination(3,"over_18","str","fact")
        self.add_destination(4,"subreddit","str","fact")

        self.add_transform(1,'CLEAN_STRING')
        self.add_transform(2,'CAPITAL_LETTER')
        self.add_transform(3,'CAPITAL_STRING')

 

        self.add_mapping(1,1,1,2)
        self.add_mapping(2,2,2,1)
        self.add_mapping(3,3,3,2)
        self.add_mapping(4,4,4,3)



    def add_source(self,id, field,ty):
        if ty not in ["str"]:
          print("Error source.")
        self.db["source"].append( {"id": id,
        "source_field_name":field ,
          "source_field_mapping": "$."+field,
          "source_field_type": ty, # use python types
          "is_required": True})
       

    def add_destination(self,id, field,ty,table):
        if ty not in ["bool","str"]:
          print("Error destination.")
        self.db["destination"].append(
            {         "id": id,
                      "destination_field_name": field,
                      "destination_field_mapping": field,
                      "destination_field_type": ty,
                      "destination_field_table": table,
                      "default_value": "n/a",
                  })
        
    def add_transform(self,id, transform_mask):
        self.db["transform"].append({
                      "id": id,
                      "transform_mask": transform_mask
                  })
        

    def add_mapping(self,id, mapping_source,mapping_destination,mapping_transform):
        self.db["mapping"].append({
                      "id": id,
                      "mapping_source": mapping_source,
                      "mapping_destination": mapping_destination,
                      "mapping_transform": mapping_transform
                  })



    @property 
    def get_data_source_target_mapping(self):
        return self.db 


  

### Source class

Inherited from Interface for the common methods and from Database for common variables

In [352]:
class Source(Interface, Database):
    def __init__(self):
        Database.__init__(self)
       
    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("source")

    
    def get_data_by_id(self, id):
      self.id = id
      data = self.get
      for x in data:
          if x.get("id").__str__() == self.id.__str__():
              return x
      return None

 


### Target class

Inherited from Interface for the common methods and from Database for common variables

In [353]:
class Target(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    # # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("destination")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

### Transform Class

Inherited from Interface for the common methods and from Database for common variables

In [354]:
class Transform(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("transform", [])

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

### Mapping class

Inherited from Interface for the common methods and from Database for common variables

In [355]:
class Mappings(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("mapping")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

    def get_data_by_field(self, field_name):
        return None

### Format Class - JSON

Search the source data value inside a JSON file 

In [356]:
class JsonQuery:
    def __init__(self, json_path, json_data):
        self.json_path = json_path
        self.json_data = json_data

    def get(self):
        jsonpath_expression = parse(self.json_path)
        match = jsonpath_expression.find(self.json_data)
        source_data_value = match[0].value
        return source_data_value

### Combine it All - STTM

In [357]:
class STTM:
    def __init__(self, input_json):
        self.input_json = input_json
        self.mapping_instance = Mappings()
        self.source_instance = Source()
        self.destination_instance = Target()
        self.transform_instance = Transform()
        self.look_up_mask = {i.name: i.value for i in TransformMask}
        self.json_data_transformed = {}

    def _get_mapping_data(self):
        return self.mapping_instance.get

    def _get_mapping_source_data(self):
        return self.source_instance.get

    def get_transformed_data(self):

        for mappings in self._get_mapping_data():

            """fetch the source mapping """
            mapping_source_id = mappings.get("mapping_source")
            mapping_destination_id = mappings.get("mapping_destination")
            mapping_transform_id = mappings.get("mapping_transform")

            mapping_source_data = self.source_instance.get_data_by_id(id=mapping_source_id)
            transform_data = self.transform_instance.get_data_by_id(id=mapping_transform_id)

            """Fetch Source  field Name"""
            source_field_name = mapping_source_data.get("source_field_name")

            """if field given is not present incoming json """
            if source_field_name not in self.input_json.keys():
                if mapping_source_data.get("is_required"):
                    raise Exception(
                        "Alert ! Field {} is not present in JSON please FIX mappings ".format(source_field_name))
                else:
                    pass

            else:
                source_data_value = JsonQuery(
                    json_path=mapping_source_data.get("source_field_mapping"),
                    json_data=self.input_json
                ).get()

                """check the data type for source if matches with what we have """
                if mapping_source_data.get("source_field_type") != type(source_data_value).__name__:
                    if source_data_value is not None:
                        _message = (
                            "Alert ! Source Field :{} Datatype has changed from {} to {} ".format(source_field_name,
                                                                                                  mapping_source_data.get(
                                                                                                      "source_field_type"),
                                                                                                  type(
                                                                                                      source_data_value).__name__))
                        print(_message)
                        raise Exception(_message)

                """Query and fetch the Destination | target """
                destination_mappings_json_object = self.destination_instance.get_data_by_id(
                    id=mappings.get("mapping_destination"))

                destination_field_name = destination_mappings_json_object.get("destination_field_name")
                destination_field_type = destination_mappings_json_object.get("destination_field_type")
                # destination_field_table = destination_mappings_json_object.get("destination_field_table")
                
                
                # self.json_data_transformed["table"]=destination_field_table

                dtypes = [str, float, list, int, set, dict]

                for dtype in dtypes:

                    """Datatype Conversion """
                    if destination_field_type == str(dtype.__name__):

                        """is source is none insert default value"""
                        if source_data_value is None:
                            self.json_data_transformed[destination_field_name] = dtype.__call__(
                                destination_mappings_json_object.get("default_value")
                            )

                        else:
                            """check if you have items to transform"""
                            if transform_data is not None:
                                """ check for invalid mask name """
                                if transform_data.get("transform_mask") not in list(self.look_up_mask.keys()):
                                    raise Exception(
                                        f"Specified Transform {transform_data.get('transform_mask')} is not available please select from following Options :{list(self.look_up_mask.keys())}")
                                else:
                                    mask_apply = self.look_up_mask.get(transform_data.get("transform_mask"))
                                    converted_dtype = dtype.__call__(source_data_value)
                                    mask = f'converted_dtype{mask_apply}'
                                    curated_value = eval(mask)
                                    self.json_data_transformed[destination_field_name] = curated_value

                            else:
                                self.json_data_transformed[destination_field_name] = dtype.__call__(source_data_value)

        return self.json_data_transformed

In [358]:
transformed_data = []
for item in data:
    helper = STTM(input_json=item)
    response = helper.get_transformed_data()
    transformed_data.append(response)
    print(response)

{'title': 'Bpd And Menstrual Cycle', 'selftext': 'hi i`m 33 and have been diagnosed for 2 years. i struggle daily. but the worst is coming up to my period (sorry men) \nany women on here also find that their cycle sends their emotions off into the stratosphere? i have been acting disgustingly toward my partner, i feel as though there`s no way i can even apologise and this happens each cycle. if you do find this happens to you, have you found anything which may help?', 'over_18': 'False', 'subreddit': 'BPD'}
{'title': "It'S Getting Too Much", 'selftext': 'i just want to give into my impulses and finally let go of all self control. i just want to bash my head against a wall constantly and hurt myself. getting help is so difficult and nothing works and i just want all the pain to end. everything is so overwhelming and controlling my symptoms is exhausting and everyone turning on me for fake reasons. everything sucks', 'over_18': 'False', 'subreddit': 'BPD'}
{'title': 'Is Anyone Else Extre

In [359]:
pd.DataFrame(transformed_data)

,title,selftext,over_18,subreddit
0,Bpd And Menstrual Cycle,hi i`m 33 and have been diagnosed for 2 years....,False,BPD
1,It'S Getting Too Much,i just want to give into my impulses and final...,False,BPD
2,Is Anyone Else Extremely Fixated On Past Relat...,not sure if this is the right flair \n\nall pa...,False,BPD
3,Is There Going To Be A Time Where I Don'T Miss...,"i miss her so much, she is alive she and the ...",False,BPD
4,My Fp Has Been Acting Different And It Is Kill...,"this is my longest boyfriend i have ever had, ...",False,BPD
5,Does Anyone Else Dread Going To Work????,i work in the food industry so i have to talk ...,False,BPD
6,Help,[removed],False,BPD
7,Fixated On Social Group Of Shared House I'Ve J...,cognitively i want to be alone but feel so con...,False,BPD
8,It`S Been A While,i`ve not posted on here for a while now but i ...,True,BPD
9,Getting Emotional Whiplash In My Relationship,[deleted],False,BPD


In [360]:
pd.DataFrame(transformed_data).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      50 non-null     object
 1   selftext   50 non-null     object
 2   over_18    50 non-null     object
 3   subreddit  50 non-null     object
dtypes: object(4)
memory usage: 1.7+ KB


In [361]:


with open("data.json", "w") as file:
    json.dump(transformed_data, file)